In [1]:
from pyspark import SparkContext

sc = SparkContext (master = 'local[4]')

from pyspark.sql import * 

sqlSpark = SQLContext(sc)

In [15]:
%%time
query = '''
with q0 as (select country, count(country) as count_country
from parquet.`userdata.parquet`
group by country
order by count(country) desc)
select * from q0 where count_country > 100
'''
print (query)
userdata=sqlSpark.sql(query)
userdata.show()
print (userdata.count())
userdata_df = sqlSpark.sql(query).toPandas()


with q0 as (select country, count(country) as count_country
from parquet.`userdata.parquet`
group by country
order by count(country) desc)
select * from q0 where count_country > 100

+-------------+-------------+
|      country|count_country|
+-------------+-------------+
|        China|          932|
|    Indonesia|          462|
|       Russia|          310|
|  Philippines|          253|
|       Brazil|          195|
|     Portugal|          191|
|       Poland|          178|
|       France|          155|
|       Sweden|          132|
|United States|          103|
+-------------+-------------+

10
CPU times: user 0 ns, sys: 10 ms, total: 10 ms
Wall time: 3.94 s


In [19]:
# user defined functions (UDFs)
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
def count_Nan(V):
    A = unpackArray(V, data_type = np.float16)
    return int(sum (np.isnan(A)))

# register as udf
count_nan_udf = udf(count_Nan, IntegerType())

# apply udf
userdata_df = userdata_df.withColumn('nan_no',count_nan_udf(userdata_df.Values))

AttributeError: 'DataFrame' object has no attribute 'withColumn'